<a href="https://colab.research.google.com/github/NikolayLenkovNikolaev/SAS-in-Clinical-Trial/blob/main/B_L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Power analysis and sample size by simulations

The first question faced by a statistical consultant is:

“How many subjects (animals, units) do I need?”

The answer to this question is crucial.

It determines the feasibility of the study in terms of time and resources used, and therefore in terms of costs.

There are also important ethical consequences associated to
the sample size calculation:
- an oversized study would expose too many subjects to an experimental intervention potentially less effective than a standard intervention;
- the inclusion of few patients may not clearly reveal a
beneficial effect of the new intervention, to the detriment of future patients.

Outcome:

A key question is to define the outcome, including the specific measurement variable, the clinician/researcher has in mind as the primary outcome.

Example:

We want to investigate the efficacy of a new therapy, compared to the standard therapy, in improving the prognosis of patients with actinic keratosis.

We decide to conduct a randomized controlled superiority clinical trial on patients presenting more than 5 lesions.

Information needed to decide how many patients to recruit in the study:
1. main outcome variable of the study and type of outcome (binary? continuous? count?);
2. expected difference between the new therapy and the
standard therapy outcomes (the difference should be clinically relevant);
3. variability of the outcome measure;
4. statistical method used to assess the superiority of the new treatment.

1. main outcome variable of the study and type of
outcome (binary? continuous? count?)
- complete response: all lesions disappear at 90 days from treatment initiation (binary outcome: yes/no)
- decrease in total lesion diameter at 90 days from treatment initiation (continuous outcome, measured in mm)
- number of lesions disappeared at 90 days from treatment initiation (count outcome)


2. expected difference between the new therapy and the standard therapy outcomes (the difference should be clinically relevant)

Based on historical data, the complete response proportion
for patients treated with the standard therapy is assumed to be 60% (p1=0.6).
With the new therapy, complete response is believed to increase at 80% (p2=0.8).


3. variability of the outcome measure

With binary outcomes (Bernoulli r.v.) the expected variance of the response is determined by the success probability itself*, according to the formula [p × (1-p)].

For a patient assuming the standard treatment is: [0.6 × (1-0.6)]
For a patient assuming the new treatment is: [0.8 × (1-0.8)]

*This is not the case for continuous outcomes, such as normally distributed data, where expected outcome (mean) and its variance are independent and must be
specified.

4. statistical method used to assess the superiority of
the new treatment.
- $\chi^2$ test for a two-way contingency table, or Z test on the
difference between proportions (the two tests are
equivalent)

## Hypothesis testing:
Sample size calculation is based on the hypothesis testing
theory introduced by Neyman and Pearson in 1933.

To apply the Neyman and Pearson theory one needs to
specify:
- a null hypothesis (H_0): in most superiority RCTs the null hypothesis is assumed to be the hypothesis of no difference between treatment arms;
- a defined alternative hypothesis (H1): usually, in superiority RCTs, H1 is the expected benefit from the new treatment with respect to the standard

## Alternative hypothesis
The choice of the alternative hypothesis is challenging. Researchers sometimes say that if they knew the value of the alternative hypothesis, they would not need to do the study.

Previous example:

$H_0:$ P(new treatment) - P(standard treatment) = 60% - 60% = 0

$H_1:$  P(new treatment) - P(standard treatment) = 80% - 60% = 20 $\delta$

## Decision rule
Given H0 and H1, it is defined a threshold value of the effect above (or below) which the null hypothesis will be rejected after having conducted the study.
This threshold, as we will see, can be much greater or much lower than the expected effect defined in H1.

## Ertrros:
The threshold is a function of the sample size of the study and
the errors one is willing to tolerate.

The errors could be:
-a false positive result (i.e. to reject the null hypothesis when in
fact it is true – also know as type I error,$\alpha$ error)
-a false negative result (i.e. to accept the null hypothesis when
in fact the alternative hypothesis is true – a.k.a. type II error, $\beta$
error)

Usually, the tolerated type I error is fixed equal to 5% (twosided) or 2.5% (one-sided).

The type II error is usually fixed equal to 10% or 20%, or, equivalently, the power of the study is fixed equal to 90% or 80%.


## Simulation:

How to calculate the threshold past which the null hypothesis is rejected, in the example discussed before?
Let’s assume we don’t know the theorical distribution of the difference between two proportions, and therefore that we don’t know the formula to calculate the sample size when testing difference between proportions. Then, we conduct a
simulation study, trying to derive the threshold from the simulation results.

## Simulation with N (for each group)=10
H0 is true (the treatments are equally effective: 60% of complete responses).
Let’s simulate data from one trial under this assumption.

```
PROC FORMAT;
VALUE TRT 0="Standard" 1="New";
VALUE STATUS 0="No response" 1="Complete response";
RUN;
DATA SAMPLE;
CALL STREAMINIT(56789);
P0=0.6;
P1=0.6;
DO TRT=0 TO 1;
DO PATIENT=1 TO 10;
IF TRT=0 THEN STATUS=RAND("Bernoulli",P0);
IF TRT=1 THEN STATUS =RAND("Bernoulli",P1);
OUTPUT;
FORMAT STATUS STATUS. TRT TRT.;
END;
END;
RUN;
PROC FREQ DATA=SAMPLE;
TABLES TRT*STATUS / NOCOL NOPCT RISKDIFF;
RUN;
```

Observed difference:
70%-30%=40% (we know that the two treatments have equal effect. The difference is due to random chance)

Let’s now simulate 500 studies with the same assumptions
```
DATA SAMPLE;
CALL STREAMINIT(56789);
P0=0.6;
P1=0.6;
DO IDSAMPLE=1 TO 500;
DO TRT=0 TO 1;
DO PATIENT=1 TO 10;
IF TRT=0 THEN STATUS=RAND("Bernoulli",P0);
IF TRT=1 THEN STATUS =RAND("Bernoulli",P1);
OUTPUT;
FORMAT STATUS STATUS. TRT TRT.;
END;
END;
END;
RUN;
```

Distribution of the observed differences between proportions from the 500
simulated studies

Which thresholds allow us to reject H0 with a 2.5% (onesided) tolerated chance of obtaining a false positive?

```
PROC UNIVARIATE DATA= DIFFERENCES_RISKS;
VAR RISK;
OUTPUT OUT=THRESHOLDS PCTLPRE=P PCTLPTS=97.5;
RUN;
PROC PRINT DATA=THRESHOLDS;
RUN;
DATA THRESHOLDS;
SET THRESHOLDS;
CALL SYMPUT ("THRESHOLD",P97_5);
RUN;
%PUT &THRESHOLD;
PROC SGPLOT DATA= DIFFERENCES_RISKS;
HISTOGRAM RISK;
XAXIS LABEL="Difference between proportions" values=(-0.8 to 0.8 by 0.1);
REFLINE &THRESHOLD / AXIS=x LINEATTRS=( THICKNESS=2 COLOR=RED);
RUN;
```


The rule that ensures a 2.5% false positive rate (one-sided) is that, to reject H0,
an absolute difference between treatments higher than 40% is observed
(threshold to reject H0)


What happens when H1 is true (i.e. when the new treatment is better
than the standard treatment and the complete response probability is
20% higher)?

```
DATA SAMPLE;
CALL STREAMINIT(56789);
P0=0.6;
P1=0.8;
DO IDSAMPLE=1 TO 500;
DO TRT=0 TO 1;
DO PATIENT=1 TO 10;
IF TRT=0 THEN STATUS=RAND("Bernoulli",P0);
IF TRT=1 THEN STATUS =RAND("Bernoulli",P1);
OUTPUT;
FORMAT STATUS STATUS. TRT TRT.;
END;
END;
END;
RUN;
```

What happens when H1 is true (i.e. when the new treatment is better
than the standard treatment and the complete response probability is
20% higher)?
```
PROC FREQ DATA=SAMPLE;
TABLES TRT*STATUS / NOCOL NOPCT RISKDIFF;
BY IDSAMPLE;
ODS OUTPUT
RISKDIFFCOL1=DIFFERENCES_RISKS(WHERE=(ROW="Difference"));
RUN;
```

This is the result of one simulated study:


Observed difference:
80%-50=30%
(we know that the new treatment improves complete response probability by 20%. The remaining 10% is due to random chance)


## SImulation under $H_1$
This is the distribution of the differences from the 500 simulated trials:

How many times the threshold for rejection previously empirically
defined (40%) is exceeded given that H1 is true?

How many times the threshold for rejection previously empirically
defined (40%) is exceeded given that H1 is true?
```
DATA POWER;
SET DIFFERENCES_RISKS;
IF RISK>&THRESHOLD THEN REJECTH0=1;
ELSE REJECTH0=0;
RUN;
PROC FREQ DATA=POWER;
TABLES REJECTH0 / NOCUM BINOMIAL(LEVEL='1');
RUN;
```

Under this scenario, and using the pre-defined threshold to reject H0,
only in 50/500 studies (10%) the threshold is exceeded.
This study has therefore a low power, i.e. the type II error is high.

Note: the rejection threshold (+40%) is twice the value of the real effect of the treatment (+ 20%). Studies in which, due to random chance, the treatment has
an effect much bigger than the real one, come out as ‘positive’ studies


## How to suppress ODS Output and Graphics
To prevent thousands of pages of output from PROC FREQ, we could use ODS to
suppress the output during the computation of the contingency tables.

```
/* suppress output to ODS destinations */
ods graphics off;
ods exclude all;
ods noresults;
```

The first statement turns off ODS graphics. Technically, you only need to use this statement prior to calling a procedure (such as the REG procedure) that produces ODS graphics by default, but there is no harm in unconditionally turning off ODS graphics.

The second statement excludes all tables from open destinations such as HTML or
LISTING.

The third statement prevents ODS from making entries in the ODS Results window.

Wicklin, R. (2013). Simulating data with SAS. SAS Institute.

## Re-enable ODS output
After you compute the statistics for each BY group, re-enable ODS output by using the following statements:
```
/* re-enable output to ODS destinations */
ods graphics on;
ods exclude none;
ods results;

```

### %macro
Because these sequences of commands are used so frequently in simulation
studies, it is convenient to package them into SAS macros:
```
%macro ODSOff; /* Call prior to BY-group processing */
ods graphics off;
ods exclude all;
ods noresults;
%mend;
%macro ODSOn; /* Call after BY-group processing */
ods graphics on;
ods exclude none;
ods results;
%mend;
```

### Simulation with N (for each group)=50
Let’s increase N from 10 to 50, and simulate, as before, under H0.
```
DATA SAMPLE;
CALL STREAMINIT(56789);
P0=0.6;
P1=0.6;
DO IDSAMPLE=1 TO 500;
DO TRT=0 TO 1;
DO PATIENT=1 TO 50;
IF TRT=0 THEN STATUS=RAND("Bernoulli",P0);
IF TRT=1 THEN STATUS =RAND("Bernoulli",P1);
OUTPUT;
FORMAT STATUS STATUS. TRT TRT.;
END;
END;
END;
RUN;

%ODSOFF;
PROC FREQ DATA=SAMPLE;
TABLES TRT*STATUS / NOCOL NOPCT RISKDIFF;
BY IDSAMPLE;
ODS OUTPUT
RISKDIFFCOL1=DIFFERENCES_RISKS(WHERE=(ROW="Difference"));
RUN;
%ODSON;
PROC SGPLOT DATA= DIFFERENCES_RISKS;
HISTOGRAM RISK;
XAXIS LABEL="Difference between proportions" values=(-0.8 to 0.8 by
0.1);
RUN;
```

To obtain a 2.5% false positive results (one-sided), the rejection threshold has
lowered (the observed difference between treatments now needs to be greater
than 20%).

The empirical power (computed in a simulation under H1) still remains
too low (~50%).

### Simulation with N (for each group)=100

Let’s increase N from 50 to 100, and simulate, as before, under H0.
To obtain a 2.5% false positive results (one-sided), the rejection threshold has
lowered (the observed difference between treatments now needs to be greater
than 14%).

The empirical power (computed in a simulation under H1) is now optimal
(>80%).



## Statistical vs clinical significance
In contrast to the well-established standards for decisions regarding statistical significance, no particular guidelines exist for deciding what magnitude of difference is “clinically significant” or “clinically relevant”.

The latter decision depends upon the seriousness and the frequency of the outcome of interest and the benefit-risk-cost profile.

# 2.2. Power analysis and sample size by simulations and theory

## Thresholds for null hypothesis rejection
In many standard effect measures (such as the difference between
proportions), we do not need to empirically establish, via simulation, the
threshold for rejection of the null hypothesis (the vertical red lines in the
previous plots).
We indeed know from asymptotic theory and the central limit theorem that,
in large sample (i.e. n>50 per group), the random variable “difference
between proportions” is normally distributed, with mean = $\mu$ [the
difference between the population proportions p1 and p0], and $\sigma_{diff} = \sqrt{[p_1(1-p_1)/n_1 + p_0(1-p_0)/n_0]}$ with $n_1$ and $n_0$ the sample size in group 1 and 0 , repsectively.

Therefore, the threshold for rejection of the null hypothesis, given a onesided tolerated type I error of $\alpha /2$(for ex.0.025) will be:
$z_{1-\alpha/2} * \sqrt{[p_1(1-p_1)/n_1 + p_0(1-p_0)/n_0]} $

## Empirical power
To calculate power, we can simulate n trials under the alternative
hypothesis H1, and assess whether in each simulation the Z test* for
equality between proportions leads to the rejection of H0, given a tolerated
$\alpha$ type I error.

The proportion p of times H0 is rejected (that is, the two-sided p-value is
less than $\alpha$** or the one-sided p-value is less than $\alpha$/2) gives an empirical estimate of the study power.

Greater the number of simulations n, greater the precision of this estimate
(the empirical power).

* Z statistic for a single trial is computed as the difference between the observed proportions $\hat{p_1}$ and $\hat{p_0}$ by its asymptotic standard error $\sqrt{[\hat{p_1}(1-\hat{p_1})/n_1 + \hat{p_0}(1-\hat{p_0})/n_0]}$
- if Z > $z_{1-\alpha/2}$ then $H_0$ is rejected

** the two sided p-value for a Z test ofor eqaulity between proportions is the same as the Pearson $\chi^2$ test to evaluate associations in 2x2 table (implemented in SAS)

```
DATA SAMPLE;
CALL STREAMINIT(56789);
P0=0.6;
P1=0.8;
DO IDSAMPLE=1 TO 500;
DO TRT=0 TO 1;
DO PATIENT=1 TO 100;
IF TRT=0 THEN STATUS=RAND("Bernoulli",P0);
IF TRT=1 THEN STATUS =RAND("Bernoulli",P1);
OUTPUT;
FORMAT STATUS STATUS. TRT TRT.;
END;
END;
END;
RUN;
```

How many times the p value for the Chi-Square statistics is lower than
0.05?
```
%ODSOff;
PROC FREQ DATA=SAMPLE;
TABLES TRT*STATUS / NOCOL NOPERCENT CHISQ;
BY IDSAMPLE;
ODS OUTPUT CHISQ=PCHI;
RUN;
%ODSOn;
DATA POWER;
SET PCHI;
IF STATISTIC IN ("Chi-Square");
IF PROB<0.05 THEN REJECTH0=1;
ELSE REJECTH0=0;
RUN;
PROC FREQ DATA=POWER;
TABLES STATISTIC*REJECTH0;
RUN;
```

The empirical power is ~ 89%

## Analytical power
Since we know from asymptotic theory that the difference between
proportion under H1 is also normally distributed, we could calculate the
probability to reject H0 when the difference hypothesizes under H1 is true
(i.e. the power of the study) as:

$P(Z > Z_{1-\beta})$ with

$Z_{1-\beta}$ = (threshold value - difference under H_1) / standar deviation of the difference

These simple formulas are implemented in SAS PROC POWER.

```
PROC POWER;
TWOSAMPLEFREQ TEST=PCHI ALPHA=0.05
PROPORTIONDIFF = 0.2
REFPROPORTION = 0.6
POWER = .
NTOTAL = 100 to 400 by 20;
ODS OUTPUT OUTPUT=POWERS;
RUN;

```
Power curve with SAS PROC POWER
```

PROC SGPLOT DATA=POWERS NOAUTOLEGEND;
SERIES X=NTOTAL Y=POWER;
YAXIS MIN=0 MAX=1 LABEL="Power" GRID;
XAXIS LABEL="Total sample size" GRID;
RUN;
```

## Sample size calculation
For the most common statistical tests, analytical solutions,
based on asymptotic theory, have been proposed in order to
calculate the optimal sample size.

fig-1

In the situation of the comparison between two means (with n1=n0 and $\sigma_0^2 = \sigma_1^2$ we have:

$0 + z_{1-\alpha/2} \sigma \sqrt{2/n} = \delta - z_{1-\beta} \sigma \sqrt{2/n}$

Solving for n, we obtain

$n = \frac{2[z_{1-\alpha/2} + z_{1-\beta}]}{(\frac{\mu_0 - \mu_1}{\sigma})^s}$

In notminare:
- $\alpha = 0.05$ -> $z_{1-\alpha/2} = 1.96$
- $\beta=0.2$ => $ z_{1-\beta}= 0.84$
- $ 2(1.96 + 0.84)^2 = 15.68$

In denominatore:
- $△$ effect size, standard defference



## In the situation of the comparison between two proportions


In the situation of the comparison between two means (with n1=n0 and $\sigma_0 = \sigma_1 = \sigma = \sqrt{\bar{p}(1-\bar{p})}$ con $\bar{p} = \frac{p_1 -p_0}{2}$ we have:

$0 + z_{1-\alpha/2} \sigma \sqrt{2/n} = \delta - z_{1-\beta} \sigma \sqrt{2/n}$

Solving for n, we obtain:

$n = \frac{2[z_{1-\alpha/2} + z_{1-\beta}]}{(\frac{p_1 - p_0}{\sigma})^s}$
In notminare:
- $\alpha = 0.05$ -> $z_{1-\alpha/2} = 1.96$
- $\beta=0.2$ => $ z_{1-\beta}= 0.84$
- $ 2(1.96 + 0.84)^2 = 15.68$

In denominatore:
- $△$ effect size, standard defference


Previous example

$n = \frac{2*(1.96 + 0.84)^2}{\frac{0.8-0.6}{\sqrt{0.7*0.3}^2}} =82$ - per group, 164 total

with: $\alpha=0.05, \beta=0.02$
```
PROC POWER;
TWOSAMPLEFREQ TEST=PCHI
PROPORTIONDIFF = 0.2
REFPROPORTION = 0.6
POWER = 0.8
NTOTAL = .;
RUN;
```


https://www.stat.ubc.ca/~rollin/stats/ssize/


## Sample size calculation:
When the planned experiment is characterized by an unconventional design and/or when suitable formulas for the statistical method chosen for the analyses are not available, sample size calculation can be derived using a simulation study.

Reference:

vanBelleSampleSize.pdf

Sample_Size_basi.pdf

SampleSize_bySimulations_Landau.pdf

PowerBySimulation_STATA.pdf

## R pwr package:

```
#install.packages("pwr")
library(pwr)
pwr.2p.test(h = ES.h(p1 = 0.8, p2 = 0.6),
power=0.8,sig.level = 0.05)
```

# Esercizion:

Esercizio 1

A clinical dietician wants to compare two different diets, A and B, for diabetic
patients. She hypothesizes that diet A (Group 1) will be better than diet B (Group 2), in terms of lower blood glucose. She plans to get a random sample of diabetic patients and randomly assign them to one of the two diets. At the end of the experiment, which lasts 6 weeks, a fasting blood glucose test will be conducted on each patient. She also expects that the average difference in blood glucose measure between the two groups will be about 10 mg/dl. Furthermore, she also assumes that blood glucose is normally distributed and that the standard deviation of blood glucose distribution for both diets is 16.03.

The dietician wants to know the number of subjects needed in each group
assuming equal sized groups, a two-sided test, 5% of type I error and 20% of type II error.

a. Risolvere l’esercizio tramite uno studio di simulazione
[impostare inizialmente n da 10 a 100 by 10]

b. Risolvere l’esercizio tramite la PROC POWER

Esercizio: 2

A randomized study among patients with angina (heart chest pain) is to be
conducted with five-year follow-up. Patients are to be randomized to medical and
surgical treatment. Suppose that the estimated five-year medical mortality is 10% and it is hoped that the surgical mortality will be only half as much (5%) or less. If a test of binomial proportions at the 5% significance level is to be performed, and we want to be 90% certain of detecting a difference of 5% or more, what sample sizes are needed for the two (equal-sized) groups?

a. Risolvere l’esercizio tramite uno studio di simulazione
[impostare inizialmente n da 100 a 1000 by 50]

a. Risolvere l’esercizio tramite la PROC POWER

Es:3

La PROC POWER non considera altre statistiche chi-quadrato (quella aggiustata per continuità, basata sul rapporto verosimiglianze, di MantelHaenszel), alternative al chi-quadrato di Pearson ottenuto dalla PROC FREQ, per il confronto tra due proporzioni.

Si vuole investigare, tramite uno studio di simulazione, quale tra le statistiche chi-quadro calcolate dalla PROC FREQ e basate su una tabella 2x2 è la meno potente. Usare i dati dell’esercizio 2 per
 impostare lo studio di simulazione.